# Introduction
This code replicates the Google Colab topic modeling notebook for use on Jupyter Notebook or another local Python environment.

Below, Gensim is used to create LDA topic models of a pre-loaded dataframe of texts. Methods for evaluating topic coherence and analyzing topic output are also demonstrated. 

This code is adapted from [Intro to Topic Modeling with Gensim and pyLDAvis](https://github.com/hawc2/text-analysis-with-python/blob/master/Topic_Modeling.ipynb) and works well with input from from the Text Sectioning and Disaggregation code from [this repository](https://github.com/SF-Nexus/Extracted-Features).



# Install Packages
This code requires three main packages:
- **NLTK:** Cleaning disaggregated data
- **Gensim:** Preprocessing data and creating word embeddings, coherence models and topic models
- **LDAvis:** Visualizing topic models

Several other packages for wrangling and processing the data, such as io and pandas, will also be installed. 

In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/megankane/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [3]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

/Users/megankane/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/megankane/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [9]:
import numpy as np
import pandas as pd
import io
import os
import re
from pprint import pprint

# Retrieve and Convert Corpus to Data Frame

This code requires the upload of a previously created dataframe which contains a corpus of disaggregated texts. For optimal use, this dataframe should also contain labels associated with each text (e.g. book or chapter numbers). 

In [18]:
##Get current working directory 
path = os.getcwd()
print(path)

#Change working directory
path = os.chdir("/Users/megankane/Downloads/Extracted-Features-main 3/data")

#Upload dataframe√
df = pd.read_csv('chapters_sample.csv')
df

/Users/megankane/Downloads/Extracted-Features-main/notebooks


,Unnamed: 0,Book + Chapter,Text
0,0,1958_ALDISS_STARSHIP _Chapter_0,1 PART Quarters
1,1,1958_ALDISS_STARSHIP _Chapter_1,1 A A A A A Again All Almost Although An And A...
2,2,1958_ALDISS_STARSHIP _Chapter_2,1 2 A A A A A A A A A A Added All An An And An...
3,3,1958_ALDISS_STARSHIP _Chapter_3,3 A A A A A A Ah All Almost Although And And A...
4,4,1958_ALDISS_STARSHIP _Chapter_4,2 4 A A A A After All Always An And And And An...
...,...,...,...
77,77,1974_BAYLEY_THEFALLOFCHRONOPOLIS _Chapter_7,1 1 1 4 5 7 A A A A A After After After Ah All...
78,78,1974_BAYLEY_THEFALLOFCHRONOPOLIS _Chapter_8,1 1mperator 8 A A A A A A A A According Accord...
79,79,1974_BAYLEY_THEFALLOFCHRONOPOLIS _Chapter_9,0 1 5 5 5 6 6 6 6 9 A A A A A A A A A A A A A ...
80,80,1974_BAYLEY_THEFALLOFCHRONOPOLIS _Chapter_10,1 10 161 A A A Absol Absol Achronal Achronal A...


In [19]:
#Drop any empty cells
df = df.dropna()
df

,Unnamed: 0,Book + Chapter,Text
0,0,1958_ALDISS_STARSHIP _Chapter_0,1 PART Quarters
1,1,1958_ALDISS_STARSHIP _Chapter_1,1 A A A A A Again All Almost Although An And A...
2,2,1958_ALDISS_STARSHIP _Chapter_2,1 2 A A A A A A A A A A Added All An An And An...
3,3,1958_ALDISS_STARSHIP _Chapter_3,3 A A A A A A Ah All Almost Although And And A...
4,4,1958_ALDISS_STARSHIP _Chapter_4,2 4 A A A A After All Always An And And And An...
...,...,...,...
77,77,1974_BAYLEY_THEFALLOFCHRONOPOLIS _Chapter_7,1 1 1 4 5 7 A A A A A After After After Ah All...
78,78,1974_BAYLEY_THEFALLOFCHRONOPOLIS _Chapter_8,1 1mperator 8 A A A A A A A A According Accord...
79,79,1974_BAYLEY_THEFALLOFCHRONOPOLIS _Chapter_9,0 1 5 5 5 6 6 6 6 9 A A A A A A A A A A A A A ...
80,80,1974_BAYLEY_THEFALLOFCHRONOPOLIS _Chapter_10,1 10 161 A A A Absol Absol Achronal Achronal A...


In [20]:
#Add values in Text column to new list (for further cleaning)
data = df.Text.values.tolist()

# Clean Texts
Below are several preprocessing measures to help improve quality of text used for the topic model. Some may or may not be useful depending on specifications of corpus or goals of topic modeling:
- **Stopword Removal:** Removes globally common words from corpus which may skew topics; some stopwords may be useful for analysis and some studies show that removing more than the top 10 most common words does not significantly impact topic model results 
- **Bigrams/Trigram Modeling:** Recognizes common 2- and 3-word phrases to prevent disambiguation (e.g. spaceship vs. space, ship); may misidentify phrases especially in corpora with large/uncommon vocabularies like sci-fi
- **Lemmatization:** Transforms words to root form to aid recognition; some studies show may be redundant, unnecessary or even inaccurate in conflating differing words with same root meanings 

*Read More:*

https://maria-antoniak.github.io/2022/07/27/topic-modeling-for-the-people.html

https://towardsdatascience.com/6-tips-to-optimize-an-nlp-topic-model-for-interpretability-20742f3047e2


In [21]:
#Define list of stopwords
stop_words = stopwords.words('english')

# Add further stopwords by simply "appending" desired words to dictionary
stop_words.append('CHAPTER')

In [22]:
#Remove punctuation
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

<>:2: DeprecationWarning: invalid escape sequence \S
<>:3: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \S
<>:3: DeprecationWarning: invalid escape sequence \s
/var/folders/8k/6tq9yjnx511d5q4fkv00y_600000gn/T/ipykernel_34568/3478888187.py:2: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
/var/folders/8k/6tq9yjnx511d5q4fkv00y_600000gn/T/ipykernel_34568/3478888187.py:3: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [23]:
#Define function to perform simple preprocessing on text
def sent_to_words(sentences):
    for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [24]:
#Run processing function on texts
data_words = list(sent_to_words(data))
#print(data_words[:1])

In [25]:
#Define stopword removal
def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc))
if word not in stop_words] for doc in texts]

#Define function to make bigrams
def make_bigrams(texts):
   return [bigram_mod[doc] for doc in texts]

#def make_trigrams(texts):
#   return [trigram_mod[bigram_mod[doc]] for doc in texts]

#Define function to lemmatize texts
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
   texts_out = []
   for sent in texts:
     doc = nlp(" ".join(sent))
     texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
   return texts_out

In [26]:
#Define bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [27]:
#Run functions to remove stopwords, make bigrams, and lemmatize text
data_words = remove_stopwords(data_words)
#data_words_bigrams = make_bigrams(data_nostops)
#data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#print(data_lemmatized[:4])

In [28]:
data_words

[['part', 'quarters'],
 ['almost',
  'although',
  'another',
  'ansa',
  'anything',
  'aware',
  'bergass',
  'bergass',
  'bergass',
  'besides',
  'better',
  'beyond',
  'bob',
  'bob',
  'bob',
  'cheap',
  'cheap',
  'cheap',
  'cheap',
  'cheer',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complain',
  'complains',
  'complains',
  'complains',
  'complains',
  'complains',
  'complains',
  'complains',
  'complains',
  'consequently',
  'create',
  'daise',
  'dimly',
  'disease',
  'dont',
  'dont',
  'dont',
  'dont',
  'easy',
  'eff',
  'eff',
  'eff',
  'eff'

# Building Dictionary and Corpus
Once the dataset is cleaned, two inputs must be created to run the topic model. Creating the dictionary maps every word in the corpus with a unique id number. The variable corpus is calculated by determining the frequency of each word in the document. Another optional cleaning measure can be used here--removing words that are extremely rare (existing in < n number of texts) or common (existing in > n number of texts).  

In [29]:
# Import dictionary
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(data_words)

#Calculate term document frequency for each word in dataset
corpus = [dictionary.doc2bow(doc) for doc in data_words]
corpus

[[(0, 1), (1, 1)],
 [(0, 1),
  (1, 12),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 2),
  (26, 2),
  (27, 1),
  (28, 8),
  (29, 2),
  (30, 3),
  (31, 4),
  (32, 2),
  (33, 4),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 4),
  (39, 1),
  (40, 2),
  (41, 4),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 2),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 2),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 6),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 2),
  (73, 1),
  (74, 2),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 15),
  (79, 1),
  (80, 1),
  (81, 11),
  (82, 1),
  (83, 2),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 8),
  (89, 4)

In [ ]:
# OPTIONAL cleaning: filter out words that occur less than 20 documents, or more than 50% of the documents.
#dictionary.filter_extremes(no_below=2, no_above=0.8)

In [30]:
#Get number of unique tokens and documents
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 21109
Number of documents: 80


# Find Optimal Number of Topics

Calculating topic model coherence (i.e. similarity between the highest-scoring words in each topic) is one way to assess the optimal number of topics to use when creating and visualizing topic models. Below are two methods of calculating coherence: **C_V coherence**, which is calculated based on word co-occurences, and **U_Mass coherence**, which is calculated based on how frequently documents containing high-scoring words co-occur in the corpus. Both have been used in prior research and either may yield better results, depending on corpus specifications.

Additional readings: 

https://aclanthology.org/D12-1087.pdf 

https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html

## C_V Coherence Calculations

The function below calculates C_V coherence scores for n topic models run based on set parameters. Coherence is calculated on a range of 0 < x < 1 where higher-scoring models are assumed more coherent. For example, a model with a score of .50 is more coherent than a model with a .25 score.  

In [ ]:
#Define list for model and coherence values
coherence = []

#Find coherence for set range of models and append to list
for k in range(2,200):
    print('Round: '+str(k))
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(corpus, num_topics=k, \
               id2word = dictionary, eval_every = None)
    
    cm = gensim.models.coherencemodel.CoherenceModel(\
         model=ldamodel, texts=data_words,\
         dictionary=dictionary, coherence='c_v')   
                                                
    coherence.append((k,cm.get_coherence()))

In [ ]:
# Transpose coherence data
x, y = np.array(coherence).T
  
      
# plot our list in X,Y coordinates
optimal, = plt.plot(x, y)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()


In [ ]:
#Get coherence score for each num topics sorted from highest to lowest
#Highest value will be optimal number of topics
Data = {'Num Topics': optimal.get_xdata(), 'Coherence': optimal.get_ydata()}
type(Data)
df_optimal = pd.DataFrame.from_dict(Data)
df_optimal.sort_values(by='Coherence',ascending=False).head()

## U_Mass Coherence Calculation
The function below calculates U_Mass coherence scores of n topic models run based on set parameters. Coherence is calculated on a range of -14 < x < 14 where lower-scoring models are more coherent. For example, a model with a score of -.4 is less coherent than a model with a -.9 score. 


In [ ]:
#Define list for model and coherence values
coherence2 = []

#Find coherence for set range of models and append to list
for k in range(2,200):
    print('Round: '+str(k))
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(corpus, num_topics=k, \
               id2word = dictionary, eval_every = None)
    
    cm = gensim.models.coherencemodel.CoherenceModel(\
         model=ldamodel, texts=data_words,\
         dictionary=dictionary, coherence='u_mass')   
                                                
    coherence2.append((k,cm.get_coherence()))

In [ ]:
# Transpose coherence data
x, y = np.array(coherence2).T
        
# plot our list in X,Y coordinates
optimal2, = plt.plot(x, y)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()


In [ ]:
#Get coherence score for each num topics sorted from highest to lowest
#Lowest value will be optimal number of topics
Data = {'Num Topics': optimal2.get_xdata(), 'Coherence': optimal2.get_ydata()}
type(Data)
df_optimal2 = pd.DataFrame.from_dict(Data)
df_optimal2.sort_values(by='Coherence',ascending=True).head()

# Create Topic Models with Optimal Parameters
Input the parameters of the topic model and run. The model has multiple parameters, including: 
- num_topics: Number of topics the model will generate (default = 100)
- chunksize: Number of documents processed at a time (default = 2000)
- passes: Number of times model is trained on corpus (default = 1)
- iterations: Number of times model "loops" over each document (default = 50)

Calculating coherence (above) helps determine topic number, and other methods can be used to determine appropriate values for the other parameters. 

**Chunk size:** Setting chunk size to a larger number than that of documents in the model ensures that all documents are processed at once (though this requires enough memory space). 

**Passes and Iterations:** A common way to determine the best number of passes and iterations is by training a topic model and checking the "log" to see the document convergence rate (what percentage of topic/word assignments attain stability). If convergence is low, increase number of passes and interations. 

In general, as chunksize increases, passes and iterations should increase as well. Also keep in mind that corpus size may effect number of topics--in the cases of smaller corpora, using too many topics will likely make them too general OR too limited to the context of only one text. Consider running multiple models and comparing coherence, perplexity, and top words per topic. 

Additional reading: 

https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#12buildingthetopicmodel

In [ ]:
#Import logging to gauge passes and iterations; will output file to working directory
import logging
logging.basicConfig(filename='gensim.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.INFO)

In [31]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 1000
passes = 20
iterations = 40
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=1
)

In [32]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute C_V Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.54793404246653

Coherence Score:  0.5004856772451942


# Examine Topic Model Output
Once the model has been run, it is possible to retrieve the top words in each topic and visualing the model. These are methods can help further assess model coherence and point to future directions for analysis:
- **Top Words Per Topic:** Evaluate to what extent each topic contains semantically similar words, how/why these words might be meaningful in context of corpus
- **Visualizations:** Determine topic relatedness (how far apart topic circles are on plane) and topic prevalence (how large circles are corresponds to topic prevaence in corpus)

Additional reading: 

https://towardsdatascience.com/6-tips-to-optimize-an-nlp-topic-model-for-interpretability-20742f3047e2

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#15visualizethetopicskeywords

https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/


In [34]:
#Print n number of words in each topic
for idx, topic in lda_model.print_topics(num_words=20):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.001*"tricky" + 0.001*"pimesons" + 0.001*"humbart" + 0.001*"computer" + 0.001*"rilke" + 0.001*"girls" + 0.001*"supply" + 0.001*"data" + 0.001*"miss" + 0.001*"pencil" + 0.001*"branch" + 0.001*"electricity" + 0.001*"nickname" + 0.001*"insurance" + 0.001*"incorporated" + 0.001*"fellers" + 0.001*"imperious" + 0.000*"quantity" + 0.000*"compact" + 0.000*"nuclear"
Topic: 1 
Words: 0.013*"svoboda" + 0.010*"said" + 0.009*"coffin" + 0.007*"would" + 0.006*"could" + 0.004*"back" + 0.004*"one" + 0.004*"get" + 0.004*"like" + 0.004*"well" + 0.004*"dont" + 0.003*"must" + 0.003*"go" + 0.003*"danny" + 0.003*"know" + 0.003*"time" + 0.003*"earth" + 0.003*"wolfe" + 0.003*"im" + 0.003*"man"
Topic: 2 
Words: 0.016*"window" + 0.009*"house" + 0.008*"one" + 0.008*"side" + 0.008*"door" + 0.007*"left" + 0.007*"two" + 0.007*"could" + 0.007*"back" + 0.007*"right" + 0.006*"hand" + 0.006*"small" + 0.005*"floor" + 0.005*"garage" + 0.005*"wall" + 0.004*"light" + 0.004*"room" + 0.004*"round" + 0.004*"g

In [44]:
#Create dictionary with topics and words
my_dict = {"Topic":[],"Words":[]}
for idx, topic in lda_model.print_topics(num_words=20):
    my_dict["Topic"].append(idx)
    my_dict["Words"].append(topic)

#Convert dictionary to dataframe
topics_df = pd.DataFrame.from_dict(my_dict)
topics_df.head()

#Download dataframe as csv
topics_df.to_csv('topic_word_counts.csv', index=False)

In [ ]:
#Define word cloud function
from wordcloud import WordCloud 
def create_wordcloud(model, topic):
    text = {word: value for word, value in model.show_topic(topic)}
    wc = WordCloud(background_color="white", max_words=1000)
    wc.generate_from_frequencies(text)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title("Topic" + " "+ str(topic))
    plt.show()
    
#Ignore depreciation warnings
import warnings
warnings.filterwarnings("ignore")

#Create word clouds
for i in range(1,num_topics):
    create_wordcloud(lda_model, topic=i)

In [47]:
#Create visualization of topic model above 
%matplotlib inline
import pyLDAvis.gensim_models
vis = pyLDAvis.gensim_models.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary, mds='mmds')
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
pyLDAvis.show(vis)

In [ ]:
pyLDAvis.save_html(vis, 'vis200')

# Find Top Topics Per Document

Find the topic with highest percentage in each document in corpus. 

Additional reading: 
https://towardsdatascience.com/topic-modelling-in-python-with-spacy-and-gensim-dc8f7748bdbf

In [49]:
#Define function that retrieves dominant topic for each document and puts in dataframe
def format_topics_texts(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    text_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                text_topics_df = text_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    text_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    text_topics_df = pd.concat([text_topics_df, contents], axis=1)
    return(text_topics_df)

In [50]:
#Run dominant topic function on corpus
df_topic_texts_keywords = format_topics_texts(ldamodel=lda_model, corpus=corpus, texts=data_words)

# Format
df_dominant_topic = df_topic_texts_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,5.0,0.8877,"complain, man, back, one, could, head, woman, ...","[part, quarters]"
1,1,4.0,0.9999,"said, complain, one, marapper, ship, back, lik...","[almost, although, another, ansa, anything, aw..."
2,2,5.0,0.8311,"complain, man, back, one, could, head, woman, ...","[added, anger, another, big, butch, comforting..."
3,3,4.0,0.8365,"said, complain, one, marapper, ship, back, lik...","[ah, almost, although, away, back, bealie, bec..."
4,4,4.0,0.6323,"said, complain, one, marapper, ship, back, lik...","[always, answer, back, belatedly, beneath, ber..."
5,5,4.0,0.5073,"said, complain, one, marapper, ship, back, lik...","[although, another, bergasss, bob, bob, cables..."
6,6,4.0,0.9999,"said, complain, one, marapper, ship, back, lik...","[bergass, breaking, cautiously, certainly, cir..."
7,7,5.0,0.9780,"complain, man, back, one, could, head, woman, ...","[almost, another, another, bassit, besides, be..."
8,8,4.0,0.7396,"said, complain, one, marapper, ship, back, lik...","[already, although, back, belief, bending, ber..."
9,9,4.0,0.7198,"said, complain, one, marapper, ship, back, lik...","[according, although, although, anybody, berga..."


# Find Top Documents Per Topic
Calculate the top documents attributed to each topic in the model. 

Additional reading:

https://stackoverflow.com/questions/63777101/topic-wise-document-distribution-in-gensim-lda

https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/topic_methods.ipynb 

https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/#7.-The-most-representative-sentence-for-each-topic 

In [51]:
#Add Book + Chapter labels to dataframe for easier ID
doc_names = df['Book + Chapter']
df_topic_texts_keywords = df_topic_texts_keywords.join(doc_names)

In [52]:
# Get most representative text for each topic 
#Display setting to show more characters in column
pd.options.display.max_colwidth = 100

#Create new dataframe and group topic keywords by dominant topic column
topics_sorted_df = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_texts_keywords.groupby('Dominant_Topic')

#Sort data by percent contribution and select highest n values for each topic
for i, grp in sent_topics_outdf_grpd:
    topics_sorted_df = pd.concat([topics_sorted_df, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

In [53]:
# Reset Index of new df
topics_sorted_df.reset_index(drop=True, inplace=True)

# Format
topics_sorted_df.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text", "Text Name"]
topics_sorted_df = topics_sorted_df.reindex(columns=['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text Name", "Representative Text"])
# Show
topics_sorted_df.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text Name,Representative Text
0,1.0,0.9999,"svoboda, said, coffin, would, could, back, one, get, like, well",1961_ANDERSON_ORBITUNLIMITED _Chapter_1,"[actually, afterward, agriculture, ah, ah, alice, america, america, american, american, american..."
1,2.0,0.9999,"window, house, one, side, door, left, two, could, back, right",1968_ALDISS_REPORTONPROBABILITYA _Chapter_14,"[across, ahead, along, along, also, although, apart, arent, bastards, beneath, beyond, brighton,..."
2,3.0,1.0000,"aton, time, one, would, could, strat, empire, said, haight, like",NaN,"[achronal, achronal, achronal, achronal, achronal, advise, aged, agency, agency, agency, ah, ah,..."
3,4.0,0.9999,"said, complain, one, marapper, ship, back, like, vyann, could, us",1958_ALDISS_STARSHIP _Chapter_1,"[almost, although, another, ansa, anything, aware, bergass, bergass, bergass, besides, better, b..."
4,5.0,0.9780,"complain, man, back, one, could, head, woman, two, room, wall",1958_ALDISS_STARSHIP _Chapter_7,"[almost, another, another, bassit, besides, better, carefully, com, complain, complain, complain..."


In [54]:
#Download top doc per topic to dataframe
topics_sorted_df.to_csv('top_doc_per_topic.csv', index=False)

# Gensim Word2Vec (Work In Progress)

https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92 

https://www.shanelynn.ie/word-embeddings-in-python-with-spacy-and-gensim/

https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial/notebook

In [ ]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim

In [ ]:
#essays_grades_master['ID + Score'] = essays_grades_master['ID'].astype(str) + '_' + essays_grades_master['Portfolio Score'].astype(str)
#word2vec_test = essays_grades_master[['ID + Score', 'Text_NoHeaders']].copy()
word2vec_test = df.copy()
word2vec_test.head()

In [ ]:
#Get shape of df
word2vec_test.shape

In [ ]:
#Define spaCy function to lemmatize, remove stopwords and non-alphanumeric characters
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [ ]:
#Remove characters
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in word2vec_test['Text_NoHeaders'])
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

In [ ]:
#Put results in a new dataframe
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

In [ ]:
#Use bigrams to detect common phrases
from gensim.models.phrases import Phrases, Phraser

#Take list of list of words as input
sent = [row.split() for row in df_clean['clean']]

#Creates relevant list of phrases from all sentences
phrases = Phrases(sent, min_count=30, progress_per=10000)
bigram = Phraser(phrases)

#Transform the corpus based on the bigrams detected:
sentences = bigram[sent]

In [ ]:
#Count word frequency
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

In [ ]:
#Get most frequent words
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

In [ ]:
#Import word2vec
import multiprocessing

from gensim.models import Word2Vec

cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [ ]:
#Build word2vecmodel (check how to set parameters in tutorial)
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
#Build vocab table--digest all words, filter out unique words, do counts on them
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
#Set parameters to train the model
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
#Now that the model has been trained, make it more memory efficient
w2v_model.init_sims(replace=True)

In [ ]:
#Find most similar words to key terms in corpus
w2v_model.wv.most_similar(positive=["TERM"])

In [ ]:
#Check similarity between words
w2v_model.wv.similarity("TERM1", 'TERM2')

In [ ]:
#Check which word does not belong
w2v_model.wv.doesnt_match(['TERM1', 'TERM2', 'TERM3'])

In [ ]:
#Analogy difference
w2v_model.wv.most_similar(positive=["TERM1", "TERM2"], negative=["TERM3"], topn=3)

# Sources

**More Examples of Topic Modeling Research:**

*   http://www.digitalhumanities.org/dhq/vol/11/2/000291/000291.html
*   http://www.cs.columbia.edu/~blei/papers/Blei2011.pdf
*   https://maria-antoniak.github.io/resources/2019_cscw_birth_stories.pdf 

**More Topic Modeling Tools:**

*   https://github.com/polsci/colab-gensim-mallet/blob/master/topic-modeling-with-colab-gensim-mallet.ipynb
*   https://github.com/laurejt/authorless-tms 
*   https://colab.research.google.com/github/kldarek/skok/blob/master/_notebooks/2021-05-27-Topic-Models-Introduction.ipynb
